<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🧩 Semantic Segmentation with U-Net</h2>
    <p>
        In the previous notebook, we looked at <strong>Object Detection</strong> (YOLO), which draws a box around an object. Now, we go deeper. 
        <strong>Semantic Segmentation</strong> classifies every single pixel in the image.
    </p>
    <p>
        We will build the <strong>U-Net</strong> architecture from scratch. Originally designed for biomedical image segmentation, it remains one of the most popular architectures for pixel-level tasks.
    </p>
    <h3>Why "U-Net"?</h3>
    <p>
        The architecture looks like the letter <strong>U</strong>. It consists of two paths:
    </p>
    <ul>
        <li><strong>Contraction (Encoder):</strong> Captures the "context" (what is in the image) by downsampling. Similar to a standard CNN.</li>
        <li><strong>Expansion (Decoder):</strong> Enables precise localization (where is it) by upsampling.</li>
        <li><strong>Skip Connections:</strong> The secret sauce. We pass high-resolution features from the Encoder directly to the Decoder to preserve spatial details lost during pooling.</li>
    </ul>
    <p>
        First, let's set up our environment and ensure we are using the <strong>Apple M4 GPU</strong>.
    </p>
</div>

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

# Detect Apple Silicon MPS (Metal Performance Shaders)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple GPU (MPS backend)")
else:
    device = torch.device("cpu")
    print("⚠️ MPS not available. Using CPU")

✅ Using Apple GPU (MPS backend)


<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🧱 Step 1: The Building Block</h2>
    <p>
        The U-Net architecture repeats a specific pattern extensively: <strong>Two Convolutional Layers</strong> followed by activation functions.
    </p>
    <p>
        To keep our code clean, we will create a helper class called <span class="code-inline">DoubleConv</span>.
    </p>
    <h3>Implementation Details</h3>
    <ul>
        <li><strong>Conv2d:</strong> Kernel size of 3, stride of 1, and padding of 1 (Same Convolution). This ensures the output size remains the same as the input size.</li>
        <li><strong>BatchNorm2d:</strong> Normalizes the output of the convolution, speeding up convergence (a modern addition to the original U-Net).</li>
        <li><strong>ReLU:</strong> The activation function to introduce non-linearity.</li>
    </ul>
</div>

In [2]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            # First Conv Layer
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            
            # Second Conv Layer
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🏗️ Step 2: Assembling the U-Net</h2>
    <p>
        Now we assemble the full architecture. This requires careful management of tensor shapes.
    </p>
    <h3>The Structure</h3>
    <ul>
        <li><strong>Downs (Encoder):</strong> A list of <span class="code-inline">DoubleConv</span> blocks. After each block, we use <span class="code-inline">MaxPool2d</span> to reduce height/width by half.</li>
        <li><strong>Bottleneck:</strong> The lowest point of the "U". High features, low spatial resolution.</li>
        <li><strong>Ups (Decoder):</strong> We use <span class="code-inline">ConvTranspose2d</span> to double the image size.</li>
    </ul>
    <p>
        <strong>⚠️ Crucial Step: The Skip Connections</strong><br>
        When we upsample, we must <strong>concatenate</strong> the tensor with the corresponding tensor from the Encoder path. If the dimensions don't match perfectly (due to odd input sizes), we resize the tensor before concatenating.
    </p>
</div>

In [3]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNet, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNet (Encoder)
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNet (Decoder)
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature * 2, feature, kernel_size=2, stride=2
                )
            )
            self.ups.append(DoubleConv(feature * 2, feature))

        # The Bottleneck (lowest point of the U)
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        
        # Final output layer (1x1 conv to map to number of classes)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # 1. Run Encoder (Downsampling)
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        # 2. Run Bottleneck
        x = self.bottleneck(x)
        
        # Reverse the skip connections list to match decoder order
        skip_connections = skip_connections[::-1]

        # 3. Run Decoder (Upsampling)
        for idx in range(0, len(self.ups), 2):
            # Upsample step (ConvTranspose2d)
            x = self.ups[idx](x)
            
            # Get the corresponding skip connection
            skip_connection = skip_connections[idx // 2]

            # Handle size mismatch (if input image size wasn't perfectly divisible by 16)
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            # Concatenate along the channel axis (dim=1)
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            # Double Conv step
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)

<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🧪 Part 3: Sanity Check</h2>
    <p>
        Before we train on real data (in the Capstone project), we must ensure the math works. 
        We will pass a random tensor through the model on your M4 chip and check the output shape.
    </p>
    <p>
        <strong>Expected Behavior:</strong>
    </p>
    <ul>
        <li>Input Shape: <span class="code-inline">(Batch, 3, 160, 160)</span> (RGB Image)</li>
        <li>Output Shape: <span class="code-inline">(Batch, 1, 160, 160)</span> (Binary Mask)</li>
    </ul>
    <p>
        If the output height and width match the input, the Skip Connections and Upsampling logic are working correctly!
    </p>
</div>

In [4]:
def test_unet():
    # Create a random tensor (Batch=1, Channels=3, H=160, W=160)
    # We send it to the Apple Silicon GPU
    x = torch.randn((1, 3, 160, 160)).to(device)
    
    # Initialize model and move to GPU
    model = UNet(in_channels=3, out_channels=1).to(device)
    
    # Forward pass
    preds = model(x)
    
    print(f"Input shape:  {x.shape}")
    print(f"Output shape: {preds.shape}")
    
    assert preds.shape == (1, 1, 160, 160)
    print("✅ U-Net Sanity Check Passed!")

if __name__ == "__main__":
    test_unet()

Input shape:  torch.Size([1, 3, 160, 160])
Output shape: torch.Size([1, 1, 160, 160])
✅ U-Net Sanity Check Passed!


<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🎨 Step 4: Creating a Synthetic Dataset</h2>
    <p>
        To test our model immediately without downloading large files, we will create a <strong>Synthetic Dataset</strong> on the fly.
    </p>
    <p>
        This dataset will generate black images with random white <strong>Circles</strong> and <strong>Squares</strong>.
        The goal of the U-Net will be to segment (highlight) these shapes against the background.
    </p>
    <h3>Why Synthetic?</h3>
    <ul>
        <li><strong>Zero Download:</strong> No need to unzip files or fix broken links.</li>
        <li><strong>Instant Feedback:</strong> We know exactly what the ground truth is.</li>
        <li><strong>Perfect for Debugging:</strong> If the model can't learn this simple task, it won't learn complex real-world images.</li>
    </ul>
</div>

In [5]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2

class ShapesDataset(Dataset):
    def __init__(self, size=1000, img_size=160):
        self.size = size
        self.img_size = img_size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        # 1. Create a blank black image (H, W, 3)
        img = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
        
        # 2. Create a blank mask (H, W) - Single channel
        mask = np.zeros((self.img_size, self.img_size), dtype=np.float32)

        # 3. Randomly decide: Square (0) or Circle (1)
        shape_type = np.random.randint(0, 2)
        
        # Random position and size
        center_x = np.random.randint(30, self.img_size - 30)
        center_y = np.random.randint(30, self.img_size - 30)
        radius = np.random.randint(10, 40)

        if shape_type == 0: # Square
            # Draw white square on image
            top_left = (center_x - radius, center_y - radius)
            bottom_right = (center_x + radius, center_y + radius)
            cv2.rectangle(img, top_left, bottom_right, (255, 255, 255), -1)
            # Draw white square on mask (value 1.0)
            cv2.rectangle(mask, top_left, bottom_right, 1.0, -1)
            
        else: # Circle
            # Draw white circle on image
            cv2.circle(img, (center_x, center_y), radius, (255, 255, 255), -1)
            # Draw white circle on mask (value 1.0)
            cv2.circle(mask, (center_x, center_y), radius, 1.0, -1)

        # 4. Convert to PyTorch Tensors
        # Image: (H, W, 3) -> (3, H, W) and normalize to [0, 1]
        img = img.transpose(2, 0, 1).astype(np.float32) / 255.0
        
        # Mask: Add channel dimension (H, W) -> (1, H, W)
        mask = np.expand_dims(mask, axis=0)

        return torch.tensor(img), torch.tensor(mask)

# Create the dataset and dataloader
train_ds = ShapesDataset(size=500, img_size=160)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

print(f"✅ Generated Synthetic Dataset with {len(train_ds)} images.")

✅ Generated Synthetic Dataset with 500 images.


<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🏋️ Step 5: Training & Accuracy</h2>
    <p>
        Now we define the training loop. Since this is a binary segmentation task (Object vs Background), we use:
    </p>
    <ul>
        <li><strong>Loss Function:</strong> <span class="code-inline">BCEWithLogitsLoss</span>. This combines a Sigmoid layer and Binary Cross Entropy Loss in one class, which is more numerically stable.</li>
        <li><strong>Optimizer:</strong> <span class="code-inline">Adam</span> with a learning rate of 1e-4.</li>
        <li><strong>Metric:</strong> Pixel Accuracy. We count how many pixels were correctly classified (Background or Object) across the entire image.</li>
    </ul>
</div>

In [6]:
import torch.optim as optim

def check_accuracy(loader, model, device="cpu"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device) # Shape: (Batch, 1, H, W)
            
            # Forward pass (output is logits)
            preds = torch.sigmoid(model(x))
            
            # Convert probabilities to binary (0 or 1)
            preds = (preds > 0.5).float()
            
            # Calculate correct pixels
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            
    print(f"Got {num_correct}/{num_pixels} with accuracy {num_correct/num_pixels*100:.2f}%")
    model.train()

# 1. Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = device # Uses the MPS device defined earlier
BATCH_SIZE = 16
NUM_EPOCHS = 3

# 2. Initialize Model, Loss, Optimizer
model = UNet(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 3. Training Loop
print("🚀 Starting Training on Synthetic Data...")

for epoch in range(NUM_EPOCHS):
    loop_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)

        # Forward
        predictions = model(data)
        loss = loss_fn(predictions, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loop_loss += loss.item()

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Loss: {loop_loss/len(train_loader):.4f}")
    
    # Check accuracy at the end of each epoch
    check_accuracy(train_loader, model, device=DEVICE)

print("🎉 Training Complete!")

🚀 Starting Training on Synthetic Data...
Epoch [1/3] Loss: 0.4136
Got 12739017/12800000 with accuracy 99.52%
Epoch [2/3] Loss: 0.2989
Got 12790118/12800000 with accuracy 99.92%
Epoch [3/3] Loss: 0.2519
Got 12796951/12800000 with accuracy 99.98%
🎉 Training Complete!
